In [16]:
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
import os
from http.server import BaseHTTPRequestHandler
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [25]:
service = Service('./chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
browser = webdriver.Chrome(service=Service(),options=options)

In [26]:
urls = [
    'https://www.airindia.com/in/en/frequently-asked-questions/booking.html',
    'https://airindia.com/in/en/frequently-asked-questions/check-in.html',
    'https://www.airindia.com/in/en/frequently-asked-questions/baggage.html',
    'https://www.airindia.com/in/en/frequently-asked-questions/self-service-reaccommodation.html',
    'https://www.airindia.com/in/en/flying-returns/faqs.html',
    'https://www.airindia.com/in/en/book/travel-insurance.html',
    'https://www.airindia.com/in/en/frequently-asked-questions/cargo.html',
    'https://www.airindia.com/in/en/frequently-asked-questions/health-and-medical-assistance.html',
    'https://www.airindia.com/in/en/frequently-asked-questions/travelling-with-infants-and-children.html'
]

In [27]:
df = pd.DataFrame(columns=['category','query','solution'])

In [58]:
browser.get(urls[0])
browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
sleep(1)

soup = BeautifulSoup(browser.page_source,'html.parser')

category = soup.find('div',{'id':'abc'}).find_all('section')[1:]


for cat in category:

    subcat = cat.find_all('div')

    for sub in subcat:

        query_list = sub.find_all('div',{'class':'accordion-item'})

        for query in query_list:

            s = [cat.find('h2').text.strip()]
            q = query.find('h2').text.strip()
            a = "\n\n".join([x.text for x in query.find('div',{'class':'accordion-body'}).find_all('p')])

            row = {'category':s,'query':q,'solution':a}

            temp = pd.DataFrame(row,columns=['category','query','solution'])

            df = pd.concat([df,temp],axis=0)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185 entries, 0 to 0
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  185 non-null    object
 1   query     185 non-null    object
 2   solution  185 non-null    object
dtypes: object(3)
memory usage: 5.8+ KB


In [60]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=['index'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  179 non-null    object
 1   query     179 non-null    object
 2   solution  179 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [61]:
df.sample(5)

,category,query,solution
107,Family Pool,Can a pramukh of one family pool be a sub-memb...,"No, a pramukh of one family pool cannot be a s..."
79,Maharaja Club,Do I have to fill out any forms to become the ...,"No, once you qualify to be a member of theMaha..."
43,Baggage Policy for Infants/Children,Is there any baggage allowance for infants tra...,We have a fixed baggage allowance of 10kg/22lb...
25,Airport check-in,When does the check-in counter at the airport ...,This may vary depending upon the airport of de...
175,Infant,What documents do I need to carry when travell...,If you are travelling with your baby who is ov...


In [62]:
df.to_json("data/fulldata.json")
df.to_csv('data/fulldata.csv')